In [1]:
from Grafo import Grafo
import numpy as np

grafito = Grafo()

grafito.agregar_nodo("pepe")
grafito.agregar_nodo("luis")
grafito.agregar_nodo("solito")
grafito.agregar_arista("pepe","luis",3)
grafito.agregar_arista("luis","pepe",-3)
print(grafito)

grafito.agregar_nodo("amiguis1")
grafito.agregar_nodo("amiguis2")
grafito.agregar_arista("amiguis1","amiguis2",no_dirigido=True)
print(grafito)

{'pepe': 0, 'luis': 1, 'solito': 2}
<Nodos: ['pepe', 'luis', 'solito']
 Adyacencias:
[[ 0.  3.  0.]
 [-3.  0.  0.]
 [ 0.  0.  0.]] >
{'pepe': 0, 'luis': 1, 'solito': 2, 'amiguis1': 3, 'amiguis2': 4}
<Nodos: ['pepe', 'luis', 'solito', 'amiguis1', 'amiguis2']
 Adyacencias:
[[ 0.  3.  0.  0.  0.]
 [-3.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.]] >


In [22]:
grafito._indices.items()

dict_items([('pepe', 0), ('luis', 1), ('solito', 2), ('amiguis1', 3), ('amiguis2', 4)])

In [21]:
indices = (np.where(grafito._mat_ady[0,:].todense()!=0))[1]
[nombre for nombre,idx in grafito._indices.items() if idx in indices]


['luis']

In [ ]:
x

In [ ]:
m={"pepe":1, "gaston":4}
m["gaston"]

In [ ]:
m["popo"]=-1

In [ ]:
m

In [ ]:
list(m.values())

In [ ]:
0.0==0

In [ ]:
ts=grafito._mat_ady[:,0]

In [ ]:
ts

In [ ]:
grafito._mat_ady[:,0]=0

In [ ]:
grafito._mat_ady.todense()